以下範例示範如何使用 Paramiko 同時建立一個簡易的 SSH Server（接受 client 端傳來的指令並在本機執行）和一個 SSH Client（收到任務完成後連回 Server 執行 Windows 的 MSG 通知指令）。

### 1. 安裝 Paramiko
```bash
pip install paramiko
```

### 2. 生成伺服器金鑰

在伺服器主機上執行：
```bash
ssh-keygen -t rsa -b 2048 -f server_host_rsa.key
```
這會產生 `server_host_rsa.key`（私鑰）和 `server_host_rsa.key.pub`（公鑰）。

### 3. Server 程式 (server.py)

In [ ]:
#!/usr/bin/env python3
import socket
import threading
import sys
import subprocess
import logging
import paramiko

logging.basicConfig(level=logging.INFO)
host_key = paramiko.RSAKey(filename=sys.argv[1])


class SSHServer(paramiko.ServerInterface):
    def __init__(self):
        self.event = threading.Event()

    # 開放 Session channel
    def check_channel_request(self, kind, chanid):
        if kind == "session":
            return paramiko.OPEN_SUCCEEDED
        return paramiko.OPEN_FAILED_ADMINISTRATIVELY_PROHIBITED

    # 啟用密碼驗證
    def get_allowed_auths(self, username):
        return "password"

    def check_auth_password(self, username, password):
        # 簡單示範：固定帳密
        if username == "myuser" and password == "mypassword":
            return paramiko.AUTH_SUCCESSFUL
        return paramiko.AUTH_FAILED

    # 接收到 exec request 時觸發
    def check_channel_exec_request(self, channel, command):
        logging.info(f"收到指令：{command}")
        try:
            out = subprocess.check_output(command, shell=True)
            channel.send(out)
            channel.send_exit_status(0)
        except Exception as e:
            channel.send(str(e).encode())
            channel.send_exit_status(1)
        return True


def start_server(host="0.0.0.0", port=2200):
    sock = socket.socket()
    sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    sock.bind((host, port))
    sock.listen(100)
    logging.info(f"伺服器啟動，監聽 {host}:{port}")
    client, addr = sock.accept()
    logging.info(f"來自 {addr} 的連線")

    transport = paramiko.Transport(client)
    transport.add_server_key(host_key)
    server = SSHServer()
    transport.start_server(server=server)

    # 等待 client exec
    chan = transport.accept(20)
    if chan is None:
        logging.warning("Channel 建立失敗")
    transport.close()


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("用法：python server.py <host_key>")
        sys.exit(1)
    start_server()


> 這段程式改編自 “sample paramiko SSH server to receive commands” 範例 [Gist](https://gist.github.com/cschwede/3e2c025408ab4af531651098331cce45)。

執行方式：
```bash
python server.py server_host_rsa.key
```

### 4. Client 程式 (client.py)

In [ ]:
import paramiko

host = "YOUR_SERVER_IP"
port = 2200
username = "myuser"
password = "mypassword"

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(host, port=port, username=username, password=password)

# 任務完成後執行 Windows MSG 通知
cmd = 'msg * "任務已完成"'
stdin, stdout, stderr = client.exec_command(cmd)

# 輸出伺服器回傳（可選）
print(stdout.read().decode(), stderr.read().decode())

client.close()

> 使用 Paramiko 的 SSHClient 執行遠端命令範例 [Akamai](https://www.linode.com/docs/guides/use-paramiko-python-to-ssh-into-a-server/?utm_source=chatgpt.com)。

### 5. 執行流程說明

1. 啟動 Server：在伺服器機器執行 `python server.py server_host_rsa.key`。
  
2. Client 發起連線：任務完成時，執行 `python client.py`，透過 SSH 連回 Server。  

3. Server 收到 exec 指令：`check_channel_exec_request` 會被呼叫並執行傳入的指令（此例為 `msg * ...`）。  

4. 使用者端彈出訊息：Windows 會跳出對話框顯示「任務已完成」。

### 6. 注意事項

- 確保 Server 防火牆開啟對應的 Port（範例為 2200）。

- 執行 `msg` 指令需具備對應權限（在 Windows 上通常要以系統或 Administrator 身份執行 SSH Server）。

- 若需更安全的身分驗證，可改用公鑰認證，只要覆寫 `check_auth_publickey` 並在 `get_allowed_auths` 回傳 `'publickey'` 即可。

- Paramiko Server 功能適合測試或內部工具，正式環境建議使用 OpenSSH 等成熟方案。


---
以上範例可作為骨架，依需求再擴充錯誤處理、並發連線或更完善的驗證機制。祝順利！